<a href="https://colab.research.google.com/github/Domdec32/IMLO/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [37]:
ImageWidth = 200
ImageHeight = 200

In [38]:
transform = transforms.Compose([
    transforms.Resize((ImageHeight, ImageWidth)),
    transforms.ToTensor()
])

In [39]:

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=transform,
)

test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform=transform,
)

evaluation_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transform,
)

In [40]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
evaluation_dataloader = DataLoader(evaluation_data, batch_size=batch_size, shuffle=False)

In [41]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 200, 200])
Shape of y: torch.Size([64]) torch.int64


In [42]:
class CNN(nn.Module):
    def __init__(self, ImageWidth, ImageHeight, num_classes=102):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

        self.fc1 = nn.Linear(in_features=128*int((ImageWidth//8))*int((ImageHeight//8)), out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.dropout(self.relu(self.fc1(x.reshape(x.shape[0], -1))))
        x = self.fc2(x)

        return x

model = CNN(ImageWidth, ImageHeight, num_classes=102)

In [43]:
x = torch.randn(32, 3, 200, 200)
y = model(x)
print(y.shape)

torch.Size([32, 102])
